In [4]:
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark.sql.types import StructType, StructField, LongType, StringType, DoubleType
from time import sleep

sparkConf = SparkConf()
sparkConf.setMaster("spark://spark-master:7077")
sparkConf.setAppName("Lab9_2")
sparkConf.set("spark.driver.memory", "2g")
sparkConf.set("spark.executor.cores", "1")
sparkConf.set("spark.driver.cores", "1")
# create the spark session, which is the entry point to Spark SQL engine.
spark = SparkSession.builder.config(conf=sparkConf).getOrCreate()

dataSchema = StructType(
        [StructField("Arrival_Time", LongType(), True),
         StructField("Creation_Time", LongType(), True),
         StructField("Device", StringType(), True),
         StructField("Index", LongType(), True),
         StructField("Model", StringType(), True),
         StructField("User", StringType(), True),
         StructField("gt", StringType(), True),
         StructField("x", DoubleType(), True),
         StructField("y", DoubleType(), True),
         StructField("z", DoubleType(), True)
         ])

# Read from a source 
sdf = spark.readStream.schema(dataSchema).option("maxFilesPerTrigger", 1) \
        .json("/home/jovyan/data/activity-data")

# Do a calculation
activityCounts = sdf.groupBy("gt").count()

# Write to a sink - here, the output is the console. 
activityQuery = activityCounts.writeStream.queryName("activity_counts") \
                    .format("console").outputMode("complete") \
                    .start()
try:
    activityQuery.awaitTermination()
except KeyboardInterrupt:
    activityQuery.stop()
    # Stop the spark context
    spark.stop()
    print("Stoped the streaming query and the spark context")

Stoped the streaming query and the spark context


In [5]:
# Stop the spark context
spark.stop()